<a href="https://colab.research.google.com/github/aas161/NeuroEvo-SANE/blob/main/SANE_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np
import csv
import os
import sys
from keras.datasets import mnist
from tensorflow import keras
from sklearn.preprocessing import normalize
from sklearn.metrics import log_loss, f1_score
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import matplotlib.pyplot as plt

In [12]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

num_classes = 10

x_train = np.reshape(x_train, (x_train.shape[0], -1)) / 255
x_test = np.reshape(x_test, (x_test.shape[0], -1)) / 255
x_val = np.reshape(x_val, (x_val.shape[0], -1)) / 255

y_train_cat = keras.utils.to_categorical(y_train, num_classes)
y_test_cat = keras.utils.to_categorical(y_test, num_classes)
y_val_cat = keras.utils.to_categorical(y_val, num_classes)

In [13]:
print("Training Images:", x_train.shape)
print("Testing Images:", x_test.shape)
print("Validation Images:", x_val.shape)
print("Training Labels:", y_train.shape)
print("Test Labels:", y_test.shape)
print("Validation Labels:", y_val.shape)

Training Images: (48000, 784)
Testing Images: (10000, 784)
Validation Images: (12000, 784)
Training Labels: (48000,)
Test Labels: (10000,)
Validation Labels: (12000,)


In [25]:
# Число эпох
epochs = 30
# Общее количество нейронов в популяции
total_neurons = 2000
# Общее количество комбинаций нейронов
total_blueprints = 500
# Количество скрытых нейронов в нейронной сети
hidden_neurons = 100
# Количество связей у нейрона
neuron_connections = 7

# Количество нейронов входного слоя
input_neurons = x_train.shape[1]
# Количество нейронов выходного слоя
output_neurons = num_classes

model_id = 1

In [27]:
class Network:
    def __init__(self, neural_network_neurons, n_input_neurons, n_output_neurons, n_hidden_neurons):

        # Количество нейронов входного слоя
        self.n_input_neurons = n_input_neurons
        # Количество нейронов выходного слоя
        self.n_output_neurons = n_output_neurons
        # Количество нейронов скрытого слоя
        self.n_hidden_neurons = n_hidden_neurons
        # Список нейронов входящих в нейронную сеть
        self.neural_network_neurons = neural_network_neurons

    def forward(self, x):
        W1, W2 = self.create_weight_matrix()
        return self.SoftMax(np.matmul(self.ReLU(np.matmul(x, W1)), W2))

    def ReLU(self, x):
        for i in range(x.shape[0]):
            for j in range(x.shape[1]):
                if x[i, j] < 0:
                    x[i, j] = 0
        return x

    def SoftMax(self, x):
        out = np.zeros(x.shape)
        for i in range(x.shape[0]):
            for j in range(x.shape[1]):
                out[i, j] = np.exp(x[i, j]) / np.sum(np.exp(x[i]))
        return out

    def create_weight_matrix(self):
        W1 = np.zeros((self.n_input_neurons, self.n_hidden_neurons))
        W2 = np.zeros((self.n_hidden_neurons, self.n_output_neurons))

        # Заполнение матрицы весов
        for i in range(0, self.n_hidden_neurons):
            hn_connections = self.neural_network_neurons[i]
            for j in range(0, len(hn_connections), 2):
                """
                Если ID нейрона относится к входному слою
                (меньше количества нейронов во входном слое),
                то заполняется матрицы весов скрытого слоя
                """
                if (hn_connections[j] < self.n_input_neurons):
                    in_n_id = int(hn_connections[j])
                    W1[in_n_id, i] = hn_connections[j+1]
                else:
                    out_n_id = int(hn_connections[j] - self.n_input_neurons)
                    W2[i, out_n_id] = hn_connections[j+1]
        return W1, W2